In [35]:
from database.db_config import current_db as db

In [36]:
#  Alter the headers types, before querying.
db.execute_ddl('ALTER TABLE "All continents updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer, ALTER COLUMN "scrap_date" SET DATA TYPE date USING to_date(cast("scrap_date" as TEXT), \'yyyy-mm-dd\')')
db.execute_ddl('ALTER TABLE "All countries updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer,ALTER COLUMN "scrap_date" SET DATA TYPE date USING to_date(cast("scrap_date" as TEXT), \'yyyy-mm-dd\')')


DDL executed successfully.
DDL executed successfully.


In [37]:
countries_minMax = db.sql_query('SELECT "Country", "TotalCases" FROM "All countries updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All countries updated" LIMIT 1),'
						                       '((SELECT MAX("TotalCases") FROM "All countries updated"LIMIT 1)))')
print(countries_minMax)

+---------+------------+
| Country | TotalCases |
+---------+------------+
|   USA   |  17074294  |
| Vanuatu |     1      |
+---------+------------+


In [38]:
# TotalCases Min and Max by a continent.
continents_minMax = db.sql_query('SELECT "Continent", "TotalCases" FROM "All continents updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All continents updated" WHERE "Continent" <>  \'World\' LIMIT 1),'
						                       '((SELECT MAX("TotalCases") FROM "All continents updated" WHERE "Continent" <>  \'World\'LIMIT 1)))')
print(continents_minMax)

+-----------+------------+
| Continent | TotalCases |
+-----------+------------+
|   Europe  |  20194352  |
|  Oceania  |   46722    |
+-----------+------------+


In [39]:
# Top 3 countries with the highest number of cases by Continents.
top3_totalCases = db.sql_query('SELECT * '
                                'FROM (SELECT "Country","Continent","TotalCases",RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "TotalCases_Rank","ActiveCases","SeriousCritical"'
                                'FROM (SELECT "Country_id","Continent_id"'
                                        'FROM "All Countries") AS t1 JOIN  (SELECT * '
                                                                            'FROM "All Continents") AS t2 USING("Continent_id")'
                                        'JOIN  (SELECT "Country","Country_id","TotalCases","ActiveCases","SeriousCritical"'
                                                'FROM "All countries updated" ) AS t3 USING("Country_id")) AS "t1"'
                                'WHERE "TotalCases_Rank" <=3 ')

print(top3_totalCases)

+------------------+---------------+------------+-----------------+-------------+-----------------+
|     Country      |   Continent   | TotalCases | TotalCases_Rank | ActiveCases | SeriousCritical |
+------------------+---------------+------------+-----------------+-------------+-----------------+
|   South Africa   |     Africa    |   866127   |        1        |    79930    |       546       |
|     Morocco      |     Africa    |   403619   |        2        |    33997    |       1078      |
|      Egypt       |     Africa    |   122086   |        3        |    10011    |        90       |
|      India       |      Asia     |  9932842   |        1        |    333048   |       8944      |
|      Turkey      |      Asia     |  1898447   |        2        |    220375   |       5988      |
|       Iran       |      Asia     |  1123474   |        3        |    237528   |       5704      |
|      Russia      |     Europe    |  2707945   |        1        |    510367   |       2300      |


In [40]:
# Countries that are located in Oceania continent.
oceania_countries = db.sql_query('SELECT "Country" '
                                 'FROM "All Countries" '
                                 'WHERE "Continent_id" IN (SELECT "Continent_id" '
                                                            'FROM "All Continents" '
                                 '                           WHERE "Continent" = \'Oceania\')')
print(oceania_countries)

+------------------+
|     Country      |
+------------------+
|    Australia     |
|   New Zealand    |
| French Polynesia |
| Papua New Guinea |
|       Fiji       |
|  New Caledonia   |
+------------------+


In [41]:
# The count of countries in each continent.
countries_perContinent = db.sql_query('SELECT "Continent","Countires_count" FROM (SELECT * FROM "All Continents") AS t1 '
                                      'JOIN (SELECT "Continent_id", COUNT("Country") AS "Countires_count" '
                                      'FROM "All Countries" G'
                                      'ROUP BY "Continent_id") AS t2 '
                                      'USING("Continent_id") '
                                      'ORDER BY "Countires_count" DESC')
print(countries_perContinent)

+---------------+-----------------+
|   Continent   | Countires_count |
+---------------+-----------------+
|     Africa    |        57       |
|      Asia     |        49       |
|     Europe    |        48       |
| North America |        39       |
| South America |        14       |
|    Oceania    |        6        |
+---------------+-----------------+


In [42]:
# The country with the highest "TotalCases" in each continent.
country_high_perContinent = db.sql_query('SELECT "Country","Continent","TotalCases" '
                                         'FROM (SELECT "Country","Continent","TotalCases" ,RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "rank" '
                                                'FROM (SELECT "Country","Country_id","TotalCases" '
                                                       'FROM "All countries updated" ) AS t1  '
                                                       'JOIN (SELECT "Country_id", "Continent_id" FROM "All Countries") AS t2 '
                                                        'USING("Country_id") '
                                                        'JOIN (SELECT "Continent_id","Continent" FROM "All Continents") AS t3 '
                                                        'USING("Continent_id")) AS t4 '
                                         'WHERE "rank" =1;')
print(country_high_perContinent)

+--------------+---------------+------------+
|   Country    |   Continent   | TotalCases |
+--------------+---------------+------------+
| South Africa |     Africa    |   866127   |
|    India     |      Asia     |  9932842   |
|    Russia    |     Europe    |  2707945   |
|     USA      | North America |  17074294  |
|  Australia   |    Oceania    |   28045    |
|    Brazil    | South America |  6970034   |
+--------------+---------------+------------+


In [43]:
# Top 5 countries with active cases.
top5_countries_active = db.sql_query('SELECT "Country","ActiveCases" '
                                     'FROM "All countries updated" '
                                     'WHERE "ActiveCases" IS NOT NULL '
                                     'ORDER BY "ActiveCases" DESC LIMIT 5;')
print(top5_countries_active)

+------------------------+-------------+
|        Country         | ActiveCases |
+------------------------+-------------+
|          USA           |   6807127   |
|         France         |   2153288   |
|         Brazil         |    771150   |
|         Italy          |    667303   |
|        Belgium         |    549457   |
|         Russia         |    510367   |
|        Ukraine         |    370734   |
|        Germany         |    351526   |
|         India          |    333048   |
|         Poland         |    244389   |
|         Serbia         |    243279   |
|          Iran          |    237528   |
|         Turkey         |    220375   |
|         Mexico         |    213922   |
|        Hungary         |    195411   |
|       Argentina        |    122061   |
|         Greece         |    112598   |
|       Indonesia        |    93662    |
|        Bulgaria        |    90128    |
|        Romania         |    87010    |
|      South Africa      |    79930    |
|         Canada

In [44]:
# The date that israel had the highest value of new cases.
israel_newCases_date = db.sql_query('SELECT "scrap_date" AS "max_NewCases_date" '
                                    'FROM "Israel" '
                                    'WHERE "NewCases" = (Select MAX("NewCases") '
                                    '                    FROM "Israel")')
print(israel_newCases_date)

+-------------------+
| max_NewCases_date |
+-------------------+
|     2020-09-27    |
+-------------------+


In [45]:
# Ranking each continent by its active cases.
continent_active_rank = db.sql_query('SELECT "Continent","ActiveCases", RANK() OVER(ORDER BY "ActiveCases" DESC) '
                                     'FROM "All continents updated" '
                                     'WHERE "Continent" <> \'World\'')
print(continent_active_rank)

+---------------+-------------+------+
|   Continent   | ActiveCases | rank |
+---------------+-------------+------+
|     Europe    |   10207947  |  1   |
| North America |   7285598   |  2   |
|      Asia     |   1480985   |  3   |
| South America |   1064758   |  4   |
|     Africa    |    312128   |  5   |
|    Oceania    |    12453    |  6   |
+---------------+-------------+------+


In [46]:
israel_minMax_dates = db.sql_query('SELECT MIN("scrap_date") as "Earliest Date",MAX("scrap_date") as "Latest Date" '
                                   'FROM "Israel"')
print(israel_minMax_dates)

+---------------+-------------+
| Earliest Date | Latest Date |
+---------------+-------------+
|   2020-08-02  |  2020-12-15 |
+---------------+-------------+
